In [ ]:
import json 
from pat_back import PatJob, AppSettings

# Submit Job

In [ ]:
job_para = json.load(open('data/pat_job_test.json'))
job_para

# Run Sequential Code 

In [ ]:
%%time

job = PatJob(job_para) 
if job.job_id> 0:
    job.perform_analysis()    

ret = PatJob.get_results(job.job_id)
res = None
if ret is not None:
    res = pd.DataFrame(json.loads(ret))


# Run with Service

In [ ]:
import requests
import uuid
import json

# files = {'para': ('pat_para', open('data/pat_job_test.json','rt'))}
job_para = json.load(open('data/pat_job_test.json'))
job_para['job_guid'] = str(uuid.uuid4())
job_para['job_name'] = 'AIG Test no correction_t2'

# files = {
#         'para': json.dumps(job_para),
#         # 'data': open(r'C:\_Working\PAT_20201019\debug\pat.valid.zip','rb')
#     }

# ret = requests.post("http://frkcat-ws10:8002/api/Jobs", files=files)
ret = requests.post("http://frkcat-ws10:8002/api/Jobs", json=job_para)
# ret = requests.post("http://localhost:5000/api/Jobs", json=job_para)
ret.status_code, ret.content

# Batch submit

In [ ]:
import requests
import uuid
import json

job_para = {
  "server": "FRKCAT-RMS4SQL1",
  "edm": "Chubb_CLNA_BUILD_20210701_EDM_v181",
  "rdm": "Chubb_CLNA_2021_FireGELwPol_RDMv181",
  "portinfoid": 7,
  "perilid": 2,
  "analysisid": 31,
  "type_of_rating": "PSOLD",
  "peril_subline": "All Perils",
  "coverage": "Building + Contents + Time Element",
  "loss_alae_ratio": 1,
  "average_accident_date": "10/1/2022",
  "trend_factor": 1.035,
  "additional_coverage": 2,
  "deductible_treatment": "Retains Limit",
  "valid_rules": -1,
  "job_name": "Chubb_2021_MedRisk",
  "user_name": "Chengyou",
  "user_email": "chengyou.xiao@guycarp.com",
  "job_guid": "2064e7f3-2292-4944-aae6-7b8d23ef3507"
}

lst=[('NACI',8,5,9),
('MedRisk',7,2,31),
('Advantage',2,5,3),
('CanadaCombined',4,5,5),
('FARM_HUREQ',5,2,29),
('WSGBF',14,5,13),
('WSGIM',15,5,15),
('SCI',13,5,11),
('WSGPD',16,5,17),
('CM_Warehouse',18,5,21),
('OMC',17,5,19),
('CM_IM',19,5,23),
('Programs',30,5,27),
('MAP',6,5,7),
('BDA',29,5,25),
('ACEIM',1,5,2)]

for port in lst:
    job_para['job_guid'] = str(uuid.uuid4())
    job_para['job_name'] = f'Chubb_2021_{port[0]}'
    job_para['portinfoid'] = port[1]
    job_para['perilid'] = port[2]
    job_para['analysisid'] = port[3]

    ret = requests.post("http://frkcat-ws10:8002/api/Jobs", json=job_para)
    print(port[0],ret.status_code, ret.content)


# Results Summary

In [ ]:
import requests
import pandas as pd

job_id = 100000196
actions = {
    'list': 'api/Jobs',
    'para': f'api/Jobs/{job_id}/Para',
    'summary': f'api/Jobs/{job_id}',
    'result': f'api/Jobs/{job_id}/Result',
    'validation': f'api/Jobs/{job_id}/Validation',
}

ret = requests.get(f"http://frkcat-ws10:8002/{actions['list']}")
try:
    df = pd.DataFrame(ret.json())
except:
    df = None
    print(ret.content)

df
# print(json.dumps(ret.json(), indent=4))

In [ ]:
import requests

job_id  = 100000075
requests.put(f"http://localhost:5000/api/public-job/{job_id}")

In [ ]:
df.groupby(by='LocationIDStack').agg({'Premium':'sum'}) 

In [ ]:
import requests

requests.post(f"http://localhost:5000/api/reset/1_2_0_3")

In [ ]:
import requests
import uuid
import json

job_para = {
    "additional_coverage": 2,
    "analysisid": 9,
    "average_accident_date": "10/1/2022",
    "coverage": "Building_Contents_BI",
    "deductible_treatment": "Retains_Limit",
    "edm": "Chubb_CLNA_BUILD_20210701_EDM_v181",
    "valid_rules": -1,
    "job_guid": "755f191e-ace9-4491-afb4-34afc6e4a10b",
    "job_name": "Chubb_2021_NACI_try",
    "loss_alae_ratio": 1,
    "peril_subline": "All_Perils",
    "perilid": 5,
    "portinfoid": 8,
    "rdm": "Chubb_CLNA_2021_FireGELwPol_RDMv181",
    "server": "FRKCAT-RMS4SQL1",
    "subject_premium": 1,
    "trend_factor": 1.035,
    "type_of_rating": "PSOLD_2016",
    "user_email": "chengyou.xiao@guycarp.com",
    "user_name": "Chengyou"
}

ret = requests.post("http://localhost:5000/api/job", json = job_para)
ret.status_code, ret.content

In [ ]:
import numpy as np

timeit()
1000 * np.power(10, np.arange(11) * 0.5)

In [ ]:
import json
import numpy as np
import pandas as pd
import pyodbc

conn_str = f'''DRIVER={{SQL Server}};Server=dfwcat-pm;Database=premium_allocation_tool;
            User Id=sa;Password=In5tr@t1;MultipleActiveResultSets=true;'''

def transform(row):
    job_id = row[0]
    user = row[1]
    js= json.loads(row[2])

    js1= {
        "job_name":js["job_name"],
        "data_source_type":js["data_source_type"]
    }

    if 'cat_data' in js: js1["cat_data"] = js['cat_data'] 
    if 'reference_job' in js: js1["reference_job"] = js['reference_job'] 
  
    if "data_correction" in js and js['data_correction']: js1["data_correction"] = js['data_correction'] 
    js1["valid_rules"] = js['valid_rules'] 
    js1["type_of_rating"] = js['type_of_rating'] 
    js1["curve_id"] = js['curve_id'] 

    if 'psold' in js: js1["psold"] = js['psold'] 
    if 'fls' in js: js1["fls"] = js['fls'] 
    if 'mb' in js: js1["mb"] = js['mb'] 

    js1["additional_coverage"] = js['additional_coverage'] 
    js1["coverage_type"] = js['coverage_type'] 
    js1["deductible_treatment"] = js['deductible_treatment'] 
    js1["loss_alae_ratio"] = js['loss_alae_ratio'] 
    js1["user_name"] = js['user_name'] 
    js1["user_email"] = js['user_email'] 
    js1["job_guid"] = js['job_guid'] 

    js= js1

    with pyodbc.connect(conn_str) as conn, conn.cursor() as cur:
        cur.execute(f"""update pat_job set parameters = '{json.dumps(js).replace("'", "''")}'
                where job_id = {job_id}""")
        cur.commit()

    print(job_id)    

with pyodbc.connect(conn_str) as conn:
    df = pd.read_sql_query("""select job_id, user_name, parameters from pat_job order by job_id""", conn)
    
df.apply(transform, axis =1)

In [ ]:
import requests

rg='_'.join([f'{a}' for a in range(100000170, 100000171)])

resp = requests.delete(f"http://localhost:5000/api/job/{rg}?req_id={req}")

print(resp.status_code, resp.content)

In [14]:

import base64
import requests
  
sample_string = str(None)
sample_string_bytes = sample_string.lower().encode("ascii")
  
base64_bytes = base64.b64encode(sample_string_bytes)
req = base64_bytes.decode("ascii")
req
resp = requests.get(f"http://localhost:5000/api/job?req_id={req}")

# len(resp.content)

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [ ]:
import requests	
requests.put(f"http://localhost:5000/api/public-job/100000140")    


In [ ]:
import requests	
requests.get(f"http://localhost:5000/api/data/100000142?data_type=results") 


In [ ]:
import numpy as np
import pandas as pd

def find_column(df, *flds):
    return next((f for f in df if f.lower() in map(str.lower, flds)), None)

df = pd.DataFrame(data=np.arange(0,10).reshape(-1, 2), columns=['A', 'B'])

find_column(df, 'b', 's', 'A1')

In [ ]:
[c for c in df if c in ['A','B', 'C']]